In [6]:
import re
import json
#import copy
import random
def replacecomma(m):        # 매개변수로 매치 객체를 받음
    n = m.group().replace(",","")    # 매칭된 문자열
    return str(n)

#infile = 'q1vol1.json'
#outfile = 'q2numbermeta.jsonl'
infile = 'q1sents.json'
#outfile = 'q2numbermetaodtsubset.jsonl'
outfile = 'q2numbermetaodtsubset.jsonl'

with open(infile, 'r') as f:
    json_data = json.load(f)

out=open(outfile,"w");


regex = r"[0-9]*\.?[0-9]+\.?[0-9]*%?"
#regex = r"[0-9]*\.?[0-9]+\.?[0-9]*"
r = re.compile(regex)
#m = r.search(t);

jsonls = []
i= 0
#for line in lines:
for d in json_data:
    i = i+1;
    if(i%1000==0):
        str(i)
    '''
    if(i>1000):
        break;
    '''    
    s=d['sent']
    #sent = d['sent']
    reg=d['pat']
    problem_no = d['problem_no']
    task_no = 2
    doc_file = d['doc_file']
    question = d['question']
    
    
    jsonl = d;
    jsonl['task_no']=2
    
    jsonl['team_id']=""
    jsonl['hash']=""

    
    label = []
    answer_auto = []
    qpoint_auto = []
    
    #숫자에서 천단위 콤마 삭제
    orgline = s;
    s = re.sub("[0-9],[0-9]",replacecomma,s);
    jsonl['text'] = s

    if(s.find(',')<0):
        continue;
    
    
    '''
    #문장 초반에(5) 콤마 나오면 삭제
    if(s.find(',')<5):
        s = s[s.find(',')+1:]
    s = s.strip();
    '''
    '''
    #문장 초반에(20) 콤마 나오면 스킵
    if(s.find(',')<20):
        continue;
    '''
    
    coms = s.split(',');
    ccnt = len(coms);
    pre = coms[0];
    
    twlen = 0
    ncnt = 0
    for com in coms[1:-1]:
        #m = r.search(com)
        #m = re.match(regex, com)
        m = re.findall(regex,com)
        
        if(m != None):
            if(len(m)>0):
                #print(str(m.groups()))
                #####print(str(m))
                ncnt = ncnt+1
        #print(com)
        ws = com.split(' ');
        wcnt = len(ws);
        twlen = twlen+wcnt
    cccnt = ccnt - 2
    avgwlen = int(twlen/cccnt) -1
    #if(avgwlen<=3):
    #if(avgwlen<=2 and avgwlen>=2):
    
    #평균 콤마사이 단어수 5 이하
    ####if(avgwlen<=4 and ncnt > 0):
    if(avgwlen<=5 and ncnt > 0):
        ####print("\n"+str(avgwlen) + " " + s+"\n")
        pass;
    else:
        continue;
    
    
    #avgwlen 기준으로 접두 생성중인데, 접두에 숫자 있는경우엔 숫자기준으로 변경 필요해보임
    #접미에도 숫자있는경우엔 숫자기준으로 변경 필요해보임
    pret = coms[0].split(' ');
    pre = ' '.join(pret[0:-1*avgwlen]);
    
    
    sut = coms[-1].split(' ');
    su = ' '.join(sut[avgwlen+1:]);
    #print("pre: "+ pre);
    #print("su: "+su+"\n");
    
    j = 0
    for com in coms:
        j = j+1
        #com에서 가장 뒤 숫자 검색
        #m = r.search(com)
        m = re.findall(regex,com)
        ansnum = ""
        if(m != None):
            if(len(m)>0):
                ansnum = m[len(m)-1]
        
        
        
        #숫자 직후 공백 추가
        if(ansnum==""):
            continue;
        if(ansnum!=""):
            com = com.replace(ansnum,ansnum+" ");
        subq = pre+"|"+com+"| "+su
        
        if(j == 1):
            subq = com+"| "+su
        if(j==len(coms)):
            subq = pre+"|"+com
        ####print("pre+com+su: "+subq )
        
        
        
        qcom = com.replace(ansnum,"");
        ansnum = ansnum.replace("%","");
        
        
        
        if(j==1):
            '''
            qcomt = qcom.split(' ');
            #qcom = qcom[len(qcom)-avgwlen:]
            #pre = ' '.join(pret[0:-1*avgwlen]);
            print(qcom)
            qcom = ' '.join(qcomt[len(qcomt)-(avgwlen):-1]);
            print(qcom)
            '''
            qcomt = com.split(' ');
            #qcom = qcom[len(qcom)-avgwlen:]
            #pre = ' '.join(pret[0:-1*avgwlen]);
            print(com)
            ####com = ' '.join(qcomt[len(qcomt)-avgwlen-1:-1*avgwlen]);
            #com = ' '.join(qcomt[-2:-1]);
            #com = ' '.join(qcomt[(-1*avgwlen)-1:]);
            com = ' '.join(qcomt[(-1*avgwlen)-2:]);
            print(com)
        
        
        subq_woans = pre+"|"+qcom+"| "+su
        
        '''
        if(j==1):
            subq_woans = qcom+"| "+su
        '''
        if(j==len(coms)):
            subq_woans = pre+"|"+qcom
            
        if(subq_woans[-1]=='.'):
            subq_woans=subq_woans[:-1]
        
        subq_woans = subq_woans+"?"

        
        
        if(com==""):
            continue;
        #start = sent.find(ansnum);
        #end = sent.find(ansnum)+len(ansnum)
        qpoint = com
        if(com.find(ansnum)>=0):
            qpoint = qpoint[0:com.find(ansnum)]
        else:
            qpoint = com
        
        if(qpoint.endswith("(")):
            qpoint = qpoint[:-1];
        if(qpoint.endswith("(-")):
            qpoint = qpoint[:-2];
        
        qpl=[]
        qpstart = s.find(qpoint);
        qpend = s.find(qpoint)+len(qpoint)
        
        qpl.append(qpstart);
        qpl.append(qpend);
        #qpl.append("qpoint")
        qpl.append("object")
        
        
        l=[]
        start = s.find(ansnum);
        end = s.find(ansnum)+len(ansnum)
        
        l.append(start)
        l.append(end)
        l.append("answer");
        
        answer_auto.append(ansnum);
        qpoint_auto.append(qpoint);
        label.append(l)
        label.append(qpl)
        
        #[0,0,'object'] 문장 맨앞 빈칸인 경우 삭제
        if(qpl[1]-qpl[0]==0):
            del label[len(label)-1]
            del qpoint_auto[len(qpoint_auto)-1]
        
        
        jsonl['answer_auto'] = answer_auto
        jsonl['object_auto'] = qpoint_auto #포기
        #낮은 버전의 doccano에서는 'labels'
        jsonl['labels'] = label
        #jsonl['label'] = label
        jsonl['meta'] = {}
        jsonl['meta']['sent']=orgline;
        jsonl['meta']['problem_no']=problem_no
        jsonl['meta']['doc_file']=doc_file
        jsonl['meta']['task_no']=task_no
        
    
        #print(subq_woans+"\t"+ansnum+"\t"+subq+"\t"+com+"\t"+s+"\t"+reg+"\n")
        
        '''
        print(pre)
        print(qcom)
        print(subq_woans+"\t"+ansnum+"\t"+subq+"\t"+com+"\t"+s+"\t"+reg+"\n")
        '''
        
        if(i%500==0):
            pass;
            ####print(pre)
            ####print(qcom)
            ####print(subq_woans+"\t"+ansnum+"\t"+subq+"\t"+com+"\t"+s+"\t"+reg+"\n")
            #print(str(i)+": "+str(jsonl)+"\n")
    
    answers_unique = list(set(answer_auto))
    if(len(answers_unique)!=len(answer_auto)):
        continue;
        
    qpoints_unique = list(set(qpoint_auto))
    if(len(qpoints_unique)!=len(qpoint_auto)):
        continue;
    
    jsonls.append(jsonl)
    
random.seed(2)
random.shuffle(jsonls)
jsontxts = []
for jsonl in jsonls:
    #jsontxts.append(json.dumps(jsonl, indent=4, ensure_ascii=False))
    jsontxts.append(json.dumps(jsonl, ensure_ascii=False))
    
c = "\n".join(jsontxts)
#c = "\n".join(jsontxts[:1000])
out.write(c);

71  고학수 편(권영준/이동진 집필 부분)
편(권영준/이동진 집필 부분)
2017서울조정1275~1285 
2017서울조정1275~1285 
239  한편
239  한편
도중진 외 3 인
도중진 외 3 인
2009년 기준으로 전체 즉결심판청구사건 중 56.8% 를 차지하는 경범죄처벌법 위반사건 중에서 주된 위반행위를 보면
위반사건 중에서 주된 위반행위를 보면
형법범 중에서는 도박죄로 인하여 청구된 인원이 13004명으로 전체 형사범 청구인원인 19488명의 66.7% 를 차지하였으며
19488명의 66.7% 를 차지하였으며
106  신동운
106  신동운
전문가협의회는 총3 회 실시되었고
전문가협의회는 총3 회 실시되었고
- 국유재산 90을 100 에 매각하기로 하고
90을 100 에 매각하기로 하고
